In [2]:
import pandas as pd
import cv2
import mediapipe as mp
import math
import numpy as np
from PIL import ImageFont, ImageDraw, Image

In [3]:
def draw_text(img, text, position, font_size, font_color):
    img_pil = Image.fromarray(img)
    draw = ImageDraw.Draw(img_pil)
    font = ImageFont.truetype("fonts/gulim.ttc", font_size)  # 'gulim.ttc' 대신 시스템에 설치된 한글 지원 폰트를 사용하세요.
    draw.text(position, text, font=font, fill=font_color)
    return np.array(img_pil)


In [4]:


# CSV 파일 로드
df = pd.read_csv('pushup_angles.csv')

# 기준 각도 설정
ideal_left_arm_angle = df['left_arm_angle'].mean()
ideal_right_arm_angle = df['right_arm_angle'].mean()



In [5]:
# 팔 각도 계산 함수
def calculate_angle(a,b,c):
    a = np.array(a) # 첫 번째 점
    b = np.array(b) # 중간 점
    c = np.array(c) # 마지막 점
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 


In [6]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
# 웹캠에서 이미지 캡처 및 포즈 추적


In [12]:
# cap = cv2.VideoCapture(0)
cap=cv2.VideoCapture("vid1.mp4")

# 웹캠 해상도를 720p로 설정
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)


#Curl counter variables
counter=0



# 운동 단계를 추적하는 변수 (예제 코드에서는 사용하지 않지만, 상태 관리를 위해 유지)
exercise_phase = "none"

 # 팔이 완전히 굽혀져 있는지 여부를 추적하는 변수
fully_bent = False

# 팔이 펴지고 있는지 여부를 추적하는 변수
extending = False

direction = "down"  # 초기 direction 값을 "up"으로 설정
# Mediapipe 포즈 모델
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # BGR 이미지를 RGB로 변환
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # 포즈 추정
        results = pose.process(image)
        
        # RGB 이미지를 BGR로 변환
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 포즈 랜드마크 추출
        try:
            landmarks = results.pose_landmarks.landmark

                        # 포즈 랜드마크 추출 및 그리기
            mp_drawing = mp.solutions.drawing_utils
            if results.pose_landmarks:
                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                        mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                        mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))
            
            # 왼쪽 팔 각도 계산을 위한 좌표
            hand_left = [landmarks[mp_pose.PoseLandmark.LEFT_INDEX.value].x, landmarks[mp_pose.PoseLandmark.LEFT_INDEX.value].y]
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            
            # 오른쪽 팔 각도 계산을 위한 좌표
            hand_right = [landmarks[mp_pose.PoseLandmark.RIGHT_INDEX.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_INDEX.value].y]
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]


                        # 몸통을 대표할 수 있는 점으로 고관절(hip) 좌표를 사용합니다
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            # 손과 전완의 정렬 확인


            # 몸통과 왼쪽 팔 사이의 각도 계산
            left_torso_angle = calculate_angle(left_shoulder, left_hip, left_elbow)

            # 몸통과 오른쪽 팔 사이의 각도 계산
            right_torso_angle = calculate_angle(right_shoulder, right_hip, right_elbow)

                        # 이제 이 좌표들을 calculate_angle 함수에 넣어 각도를 계산합니다.
            hand_forearm_alignment_left = calculate_angle(hand_left, left_wrist, left_elbow)
            hand_forearm_alignment_right = calculate_angle(hand_right, right_wrist, right_elbow)


            # 왼쪽 팔 각도 계산
            left_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
            # 오른쪽 팔 각도 계산
            right_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)


            # print(left_angle, right_angle)


            # left_angle=int(np.interp(left_angle,[-30,180],[100,0]))

            left_angle=int(np.interp(left_angle,[34,180],[100,0]))
            right_angle=int(np.interp(right_angle,[34,173],[100,0]))

            # 각도를 화면에 표시
            cv2.putText(image, str(left_angle), 
                           tuple(np.multiply(left_elbow, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            cv2.putText(image, str(right_angle), 
                           tuple(np.multiply(right_elbow, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            # 피드백 제공




   
                    
            # if left_angle>=70 and right_angle>=70:
            #     if direction==0:
            #         counter+=0.5
            #         direction=1

            # if left_angle<=70 and right_angle<=70:
            #     if direction==1:
            #         counter+=0.5
            #         direction=0    


            # 팔 각도에 따른 동작 감지 및 피드백 제공
            

            DIRECTION_CHANGE=50    
            COUNTER_CHANGE = 85
            GIVE_FEEDBACK = 80

            # 팔의 각도 점수화를 이용한 로직
            # left_score와 right_score 변수는 각각 왼쪽 팔과 오른쪽 팔의 점수화된 각도를 나타냄 (0: 완전히 펴짐, 100: 완전히 굽힘)



            # 팔의 각도를 감지하는 로직 (left_angle, right_angle은 팔의 각도를 나타냄)
            
            # if left_angle >= 10 and right_angle >= 10:  # 정규화된 각도를 기반으로 판단, 팔이 굽혀지고 있는 상태
            #     extending_fase = "down"  # 하강 단계로 설정
                
            #     if not fully_bent and left_angle >= 80 and right_angle >= 80:  # 정규화된 값으로 판단할 때의 기준값 조정
            #         fully_bent = True  # 완전히 굽혀진 상태로 표시
            #         counter += 1  # 카운트 증가
            #         extending = False  # 팔이 펴지고 있지 않음으로 설정
                    
                    
                    
                
                    
            # elif fully_bent and (left_angle <= 100 or right_angle <= 100):  # 정규화된 값으로 판단할 때의 기준값 조정
                
            #     if not extending:  # 아직 팔이 펴지고 있는 상태로 표시되지 않았다면
            #         extending_fase = "up"  # 상승 단계로 설정
                    
            #         fully_bent = False  # 완전히 굽혀진 상태를 해제
            #         extending = True  # 팔이 펴지고 있는 상태로 설정

            if left_angle >= 10 and right_angle >= 10:  # 정규화된 각도를 기반으로 판단, 팔이 굽혀지고 있는 상태
                extending_fase = "down"  # 하강 단계로 설정
    
                if not fully_bent and left_angle >= 80 and right_angle >= 80:  # 정규화된 값으로 판단할 때의 기준값 조정
                    fully_bent = True  # 완전히 굽혀진 상태로 표시
                    counter += 1  # 카운트 증가
        # extending 변수 제거

            elif fully_bent and (left_angle <= 100 or right_angle <= 100):  # 정규화된 값으로 판단할 때의 기준값 조정
                extending_fase = "up"  # 상승 단계로 설정
                fully_bent = False  # 완전히 굽혀진 상태를 해제
    # extending 변수 제거

                    


        

            # # 화면에 카운터 표시
            # cv2.putText(image, 'Counter: {}'.format(counter), 
            #             (50, 100), 
            #             cv2.FONT_HERSHEY_SIMPLEX, 1, 
            #             (255, 255, 255), 2, cv2.LINE_AA)
            
            cv2.rectangle(image, (0,0), (120,120), (255,0,0),-1)
            cv2.putText(image, str(int(counter)), (4,70), cv2.FONT_HERSHEY_COMPLEX, 1.6,
                        (0,0,255),7)
            
            leftVal=np.interp(left_angle, [0,100], [400,200])
            rightVal=np.interp(right_angle, [0,100], [400,200])

            cv2.putText(image, 'R', (24,195), cv2.FONT_HERSHEY_DUPLEX, 1.6,
                        (0,255,0),7)
            cv2.rectangle(image, (8,200), (50,400), (0,255,0), 5)
            cv2.rectangle(image, (8,int(rightVal)), (50,400), (255,0,0), -1)

            cv2.putText(image, 'L', (962,195), cv2.FONT_HERSHEY_DUPLEX, 1.6,
                        (0,255,0),7)
            cv2.rectangle(image, (952,200), (995,400), (0,255,0), 5)
            cv2.rectangle(image, (952,int(leftVal)), (995,400), (0,255,0), -1)

            if left_angle>70:
                 cv2.rectangle(image, (952,int(leftVal)), (995,400), (0,0,255), -1)
            if right_angle>70:
                 cv2.rectangle(image, (8,int(rightVal)), (50,400), (0,0,255), -1)
            # 화면에 단계 표시 (선택적)
            cv2.putText(image, 'Stage: {}'.format(extending_fase), 
                        (50, 150), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, 
                        (255, 255, 255), 2, cv2.LINE_AA)
            
            #             # 화면에 단계 표시 (선택적)
            # cv2.putText(image, 'bent: {}'.format(fully_bent), 
            #             (300, 300), 
            #             cv2.FONT_HERSHEY_SIMPLEX, 1, 
            #             (255, 255, 255), 2, cv2.LINE_AA)

            
# 피드백 제공
# 피드백 로직 개선

# 피드백 로직
            # 피드백 로직

            feedback_messages = []
            if extending_fase == "down":
                # 하강 단계일 경우
                if not fully_bent:
                    # 팔이 완전히 굽혀지지 않았으면, 팔을 더 굽혀야 함을 알림
                    feedback_messages.append("좋아요, 이제 팔을 더 굽혀주세요")
                else:
                    # 팔이 완전히 굽혀졌다면, 좋은 자세임을 피드백
                    feedback_messages.append("잘하셨습니다! 이제 팔을 완전히 펴주세요")
            elif extending_fase=="up":
                # 상승 단계일 경우
                if fully_bent and (left_angle >= 10 and right_angle >= 10):
                    # 팔이 완전히 굽혀진 상태에서 두 팔의 각도가 모두 GIVE_FEEDBACK 이상일 경우, 팔을 완전히 펴라는 피드백
                    feedback_messages.append("팔을 더 펴주세요")
                elif not fully_bent and counter>0:
                    # 팔이 완전히 굽혀진 상태가 아니거나, 둘 중 하나의 팔 각도가 GIVE_FEEDBACK보다 작을 경우, 팔을 더 펴야 함을 알림
                    feedback_messages.append("잘했어요 한번 더")

            # 예를 들어, 어깨, 엉덩이, 발목 랜드마크의 좌표를 추출
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

            # 각도 계산
            body_alignment_angle = calculate_angle(shoulder, hip, ankle)

            # 피드백 제공
            if not (175 <= body_alignment_angle <= 185):
                feedback_messages.append("등에서 다리까지 일직선이 되도록 조정해주세요.")

                        # 조건문을 사용하여 손과 전완이 일직선이 되는지 확인합니다.
            if not (160 <= hand_forearm_alignment_left <= 180 and 160 <= hand_forearm_alignment_right <= 180):
                feedback_messages.append("손이 전완과 일직선이 되도록 조정해주세요.")

                        # 피드백 메시지 화면에 표시
            for i, message in enumerate(feedback_messages):
                image = draw_text(image, message, 
                                  (50, 50 + (i * 40)), 30, (255, 255, 255))  # PIL 기반 함수로 수정
                


        except:
            pass
        
        # 화면에 결과 이미지 표시
        cv2.imshow('Mediapipe Feed', image)

        # 'q'를 누르면 종료
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


106.22224968465716 78.1249387341831
75.27457596416313 76.66844448224612
66.06110780237204 63.34568937794427
1.4854081308271176 10.579632125951415
19.133372932075932 21.25076142534154
27.07231888276958 31.284110538369237
32.328425357818396 42.88400183069254
35.223427016468364 54.30570381603042
36.566571731960565 60.21431862633738
39.71805071630602 53.77336255926864
41.41508194615574 54.53634121946066
40.69203331008627 53.29112693699283
41.50487364557211 58.193541977731975
39.095658133950025 58.96986264071051
38.44153715075417 55.67892905969509
37.253409102896605 42.982714332074224
24.3469705496395 26.476506288469903
16.893565302840113 18.740078715807705
5.613843362593869 15.989683444020487
54.17931389600275 60.43186936844012
95.01562629312816 80.80325032769082
103.267190655442 85.20808793666475
115.60199594155759 88.64253420153864
122.56402483385554 97.33278416880125
130.89847351292352 103.92967505335213
135.7447700091682 106.47484762409252
140.54124166228544 108.15307373710863
142.8486